# OGGM - data pulling

## Setting up:

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
from pyproj import Transformer
import salem
from tqdm import tqdm
import logging

from oggm import cfg, utils, workflow, tasks

cfg.initialize(logging_level='WARNING')
cfg.PARAMS['border'] = 10
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['continue_on_error'] = True
# Module logger
log = logging.getLogger('.'.join(__name__.split('.')[:-1]))

2024-08-05 12:29:14: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-08-05 12:29:14: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-08-05 12:29:14: oggm.cfg: Multiprocessing: using all available processors (N=32)
2024-08-05 12:29:14: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2024-08-05 12:29:14: oggm.cfg: Multiprocessing switched ON after user settings.
2024-08-05 12:29:14: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.


## Download OGGM data:

In [2]:
# set working directory
working_dir = '../../../data/OGGM/'
cfg.PATHS['working_dir'] = working_dir

# Set RGI version and region:
rgi_region = "11"  # Central Europe
rgi_version = "6"
rgi_dir = utils.get_rgi_dir(version=rgi_version)

In [3]:
path = utils.get_rgi_region_file(region=rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)

# We use the directories with the shop data in it: "W5E5_w_data"
base_url = "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/"
gdirs = workflow.init_glacier_directories(
    rgidf,
    from_prepro_level=3,
    prepro_base_url=base_url,
    prepro_border=10,
    reset=True,
    force=True,
)

2024-08-05 12:29:16: oggm.workflow: init_glacier_directories from prepro level 3 on 3927 glaciers.
2024-08-05 12:29:16: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 3927 glaciers


In [4]:
# Tested tasks
task_list = [
    tasks.gridded_attributes,
    tasks.gridded_mb_attributes,
    # get_gridded_features,
]
for task in task_list:
    workflow.execute_entity_task(task, gdirs, print_log=False)

2024-08-05 12:29:40: oggm.workflow: Execute entity tasks [gridded_attributes] on 3927 glaciers
2024-08-05 12:29:55: oggm.workflow: Execute entity tasks [gridded_mb_attributes] on 3927 glaciers


In [8]:
# Load PMB data:
path_PMB_GLAMOS_csv = '../../../data/GLAMOS/point/csv/'
df_pmb = pd.read_csv(path_PMB_GLAMOS_csv + 'CH_wgms_dataset.csv')
df_pmb.head(3)

,YEAR,POINT_ID,GLACIER,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,POINT_ELEVATION,POINT_BALANCE,PERIOD,RGIId
0,2003,01,oberaar,2002-10-06,2003-10-11,46.538806,8.233237,2389.812633,-6174,annual,RGI60-11.01509
1,2003,02,oberaar,2002-10-06,2003-10-11,46.536611,8.225514,2499.825727,-5310,annual,RGI60-11.01509
2,2003,03,oberaar,2002-10-06,2003-10-11,46.532136,8.207734,2679.854419,-4320,annual,RGI60-11.01509


In [9]:
# variables of interest from oggm
voi = ["aspect", "slope", "dis_from_border", "topo"]

# Initialise empty:
for var in voi:
    df_pmb[var] = np.nan

for i, row in tqdm(enumerate(df_pmb), desc='rows'):
    # Get info of that sample:
    RGI = df_pmb.iloc[i].RGIId
    POINT_LAT, POINT_LON = df_pmb.iloc[i].POINT_LAT, df_pmb.iloc[i].POINT_LON

    # Get oggm data for that RGI ID
    for gdir in gdirs:
        if gdir.rgi_id == RGI:
            break
    with xr.open_dataset(gdir.get_filepath("gridded_data")) as ds:
        ds = ds.load()

    # Transform stake coord to glacier system:
    transf = pyproj.Transformer.from_proj(salem.wgs84,
                                          gdir.grid.proj,
                                          always_xy=True)
    x_stake, y_stake = transf.transform(POINT_LON, POINT_LAT)  # x,y stake

    # Get glacier variables closest to these coordinates:
    stake = ds.sel(x=x_stake, y=y_stake, method="nearest")

    # Calculate min, max and median topography of glacier:
    # min_glacier = ds.where(ds.glacier_mask == 1).topo.min().values
    # max_glacier = ds.where(ds.glacier_mask == 1).topo.max().values
    # med_glacier = ds.where(ds.glacier_mask == 1).topo.median().values
    
    # Select variables of interest:
    stake_var = stake[voi]
    stake_var_df = stake_var.to_pandas()

    for var in stake_var_df.index:
        df_pmb.at[i, var] = stake_var_df.loc[var]
df_pmb.head()


rows: 15it [00:00, 71.87it/s]


,YEAR,POINT_ID,GLACIER,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,POINT_ELEVATION,POINT_BALANCE,PERIOD,RGIId,aspect,slope,dis_from_border,topo
0,2003,01,oberaar,2002-10-06,2003-10-11,46.538806,8.233237,2389.812633,-6174,annual,RGI60-11.01509,1.146752,0.237373,57.982758,2345.0
1,2003,02,oberaar,2002-10-06,2003-10-11,46.536611,8.225514,2499.825727,-5310,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
2,2003,03,oberaar,2002-10-06,2003-10-11,46.532136,8.207734,2679.854419,-4320,annual,RGI60-11.01509,1.183308,0.150800,295.655212,2637.0
3,2014,02,oberaar,2013-09-28,2014-09-26,46.536575,8.225487,2443.825843,-3960,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
4,2015,02,oberaar,2014-09-26,2015-09-26,46.536548,8.225526,2457.825873,-4527,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0


In [10]:
# Save to csv:
df_pmb.to_csv(path_PMB_GLAMOS_csv + 'CH_wgms_dataset_topo.csv', index=False)
df_pmb.head(3)

,YEAR,POINT_ID,GLACIER,FROM_DATE,TO_DATE,POINT_LAT,POINT_LON,POINT_ELEVATION,POINT_BALANCE,PERIOD,RGIId,aspect,slope,dis_from_border,topo
0,2003,01,oberaar,2002-10-06,2003-10-11,46.538806,8.233237,2389.812633,-6174,annual,RGI60-11.01509,1.146752,0.237373,57.982758,2345.0
1,2003,02,oberaar,2002-10-06,2003-10-11,46.536611,8.225514,2499.825727,-5310,annual,RGI60-11.01509,1.297745,0.106283,205.000000,2458.0
2,2003,03,oberaar,2002-10-06,2003-10-11,46.532136,8.207734,2679.854419,-4320,annual,RGI60-11.01509,1.183308,0.150800,295.655212,2637.0
